In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from pydub import AudioSegment as AS
from pydub.playback import play
import os
import json
import csv

In [52]:
#prepare training data
def chop_and_label(songfile, labels_to_one_hot, songs_to_label, chop):
    track_id = int(songfile.split("/")[-1].split(".")[0])
    song = AS.from_mp3(songfile)
    song_array = []
    label_array = []
    for i in range(0, int(len(song)/(chop * 1000))):
        raw = song[i*1000*chop:(i+1)*1000*chop].raw_data
        song_array.append(list(raw))
        genre_label = songs_to_label[track_id]
        label_array.append(labels_to_one_hot[genre_label])
    return song_array, label_array

songs_to_label = {}
label_to_name = {} #human readability
labels = set([])
with open("../metadata/raw_tracks.csv") as f:
    data = csv.DictReader(f)
    for d in data:
        try:
            genre = json.loads(d["track_genres"].replace("\'", "\""))[0]
        except:
            continue
        songs_to_label[int(d['track_id'])] = int(genre['genre_id'])
        label_to_name[int(genre['genre_id'])] = genre['genre_title']
        labels.add(int(genre['genre_id']))

#songs to one_hot labels
labels_to_one_hot = {}
for count, i in enumerate(labels):
    one_hot = np.zeros(len(labels))
    one_hot[count] = 1
    labels_to_one_hot[i] = one_hot
    
path = "../low_data/"
files = []
for r, d, f in os.walk(path):
    paths = [r + "/" + i for i in f if i.endswith(".mp3")]
    files += paths
songs = []
targets = []
for count, f in enumerate(files):
    s, t = chop_and_label(f, labels_to_one_hot, songs_to_label, 3)
    songs += s
    targets += t
    if count > 30:
        break



In [60]:
rand = np.random.randint(len(songs))
for count, s in enumerate(labels):
    if count == np.argmax(targets[rand]):
        print(label_to_name[s])
test_song = AS(data=bytes(songs[rand]), sample_width=2, frame_rate=17000, channels=1)
play(test_song)

Folk
